In [76]:
import ges
import sempler
import numpy as np

## Find Causal Graph

In [107]:
d = 15 # of attributes
n = 500 # of datapoints

mu_lb, mu_ub = 0, 10 # range for means of the d components
sig_lb, sig_ub = 0, 10 # range for means of the variance components

In [108]:
# Generate observational data from a Gaussian SCM using sempler
G = np.zeros((d, d))

data = sempler.LGANM(G, (mu_lb, mu_ub), (sig_lb, sig_ub)).sample(n=n)

# Run GES with the Gaussian BIC score
estimate, score = ges.fit_bic(data, phases=['forward', 'backward'])

print(estimate, score)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] 0.5597185974554577


### Choose Relationship of Interest

In [109]:
np.where(estimate>0)

(array([ 6, 13]), array([13,  6]))

In [110]:
def get_parents(x, G):
    parents = []
    for i in range(G.shape[0]):
        if(G[i, x] == 1):
            parents.append(i)
    return np.array(parents)

## Get effect size

In [121]:
# For now, predict attr 13 from 6
a = data[:, 6].reshape((n,1))
b = data[:, 13]

effect_size, resid, _, _ = np.linalg.lstsq(a, b, rcond=None)

In [123]:
effect_size[0]

0.9008889830079848